In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
import json
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################


# Exibe a resposta na tela
#display(Markdown(f"Resposta:\n {resposta.text}"))

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Vídeos --- #
##########################################
def agente_buscador(area, pratica, dominio):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Busca no goolge",
        tools=[google_search],
        instruction="""
        Como um especialista da área, busque vídeos de no mínimo 5 minutos a no máximo
        15 minutos de duração no YouTube, com profissionais brasileiros, ensinando técnicas da prática
        para melhorar determinado domínio.
        Ë importante que foco do vídeo seja a execução da prática para alcançar os objetivos.
        As práticas devem ser executados sem necessidade de auxilio de outra pessoa ou
        de equipamentos específicos.
        Os vídeos devem ser adequados para pacientes com fibromialgia, como por exemplo:
        baixa intensidade, movimentos simples e controlados, executados de forma cadenciada.
        Os vídeos devem ter pelo menos um terço do conteúdo do domínio quando o domínio for específico.
        """
    )
    entrada_do_agente_buscador = f"Área: {area}/nPrática: {pratica}/nDomínio: {dominio}"
    # Executa o agente
    videos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return videos_buscados

In [ ]:
################################################
# --- Agente 2: Selecionador de videos --- #
################################################
def agente_selecionador(area, pratica, videos_buscados):
    selecionador = Agent(
        name="agente_selecionador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista criterioso na área, com enfôque em fibromialgia.
        Dentre os vídeos encontrados selecione o vídeo em que o profissional oriente quanto
        a possibilidade de dor ou piora dos sintomas durante a prática e que explique as técnicas e posições
        com maior clareza.
        Retorne apenas 1 vídeo, o que foi escolhido.
        """,
        description="Agente que planeja posts",
    )

    entrada_do_agente_selecionador = f"Área: {area}/nPrática: {pratica}\nVídeos encontrados: {videos_buscados}"
    # Executa o agente
    video_selecionado = call_agent(selecionador, entrada_do_agente_selecionador)
    return video_selecionado

In [ ]:
######################################
# --- Agente 3: Criador de cards --- #
######################################
def agente_criador(area, pratica, video_selecionado):
    criador = Agent(
        name="agente_criador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um organizador especializado na área, com enfôque em fibromialgia.
        Faça uma descrição passo a passo de prática mostrada no vídeo escolhido.
        Divida as práticas em 2 metades iguais separaado-as por semelhança.
        Faça cards contendo o nome, a descrição da prática, número de séries e
        repetições sugeridos para cada prática. Faça uma estimativa de duração
        da execução para cada um dos 2 grupos de atividade.
        Output esperado:
        Os dados devem ser formatos em JSON com os campos: video, proficional, link, grupos.
        OS grupos devem ser formatos em JSON com os campos: nome, descricao, duracao e cards.
        Os cards devem ser formatados em JSON com os campos: nome, descricao, localizacao_no_video, series, repeticoes e duracao.
        """,
        description="Agente criador de cards de práticas"
    )
    entrada_do_agente_criador = f"Área: {area}/nPrática: {pratica}\nVídeo selecionado: {video_selecionado}"
    # Executa o agente
    cards = call_agent(criador, entrada_do_agente_criador)
    return cards

# Nova seção

In [ ]:
def get_cards(area, pratica, dominio):
# Inserir lógica do sistema de agentes ################################################
    videos_buscados = agente_buscador(area, pratica, dominio)
    video_selecionado = agente_selecionador(area, pratica, videos_buscados)
    cards = agente_criador(area, pratica, video_selecionado)
    return cards

In [ ]:
def get_scores(input_str):
    scores = json.loads(input_str)
    scores["membros"] = scores["superiores"] + scores["inferiores"]
    scores["dorsal"] = scores["lombar"] + scores["cervical"]
    scores["dor_global"] = scores["dorsal"] + scores["membros"]
    return scores

In [ ]:
def get_cards_dor (scores):
    area = "biomecânica do movimento"
    pratica = "alongamentos e yoga"
    dominio = "qualquer parte do corpo"
    if scores["dor_global"] > 12:
        pratica = "alongamentos e yoga"
        area = "fisioterapia"
    elif scores["dorsal"] > 5:
        dominio = "coluna lombar e coluna cervical"
    else:
        pratica = "exercício físico"
        if scores["membros"] > 5:
            pass
        elif scores["superiores"] > 2:
            dominio = "ombros, cotovelos e punhos"
        elif scores["inferiores"] > 2:
            dominio = "quadris, joelhos e tornozelos"
    return get_cards(area, pratica, dominio)

In [ ]:
def get_cards_sono (scores):
    area = "estudo do sono"
    pratica = "contração isométrica e respiração consciente"
    dominio = "qualquer parte do corpo"
    if scores["dor_global"] < 6:
        pratica = "relaxamento e respiração consciente"
    return get_cards(area, pratica, dominio)

In [ ]:
def get_cards_mental (scores):
    area = "psicologia"
    pratica = "mindfulness e motivacional"
    dominio = "qualquer parte do corpo"
    if scores["fadiga"] < 3 and scores["dor_global"] < 6:
       pratica = "meditação"
    if scores["memoria"] > 2:
       pratica = "mindfulness"
    return get_cards(area, pratica, dominio)

In [ ]:
print("🚀 Iniciando o Sistema de Criação de Cards com práticas para melhora da fibromialiga 🚀")

# --- Obter o Tópico do Usuário ---
input_data = """
    {
        "superiores": 4,
        "inferiores": 2,
        "lombar": 2,
        "cervical": 1,
        "insonia": 4,
        "memoria": 3,
        "fadiga": 4
    }
    """
scores = get_scores(input_data)

In [ ]:
cards_dor = get_cards_dor(scores)
# Exibir os cards gerados
print(f"Cards:\n{cards_dor}")

In [ ]:
cards_sono = get_cards_sono(scores)
# Exibir os cards gerados
print(f"Cards:\n{cards_sono}")

In [ ]:
cards_mental = get_cards_mental(scores)
# Exibir os cards gerados
print(f"Cards:\n{cards_mental}")